In [1]:
from river import datasets

dataset = datasets.Bikes()
dataset

Bike sharing station information from the city of Toulouse.

The goal is to predict the number of bikes in 5 different bike stations from the city of
Toulouse.

      Name  Bikes                                                         
      Task  Regression                                                    
   Samples  182,470                                                       
  Features  8                                                             
    Sparse  False                                                         
      Path  C:\Users\giode\river_data\Bikes\toulouse_bikes.csv            
       URL  https://maxhalford.github.io/files/datasets/toulouse_bikes.zip
      Size  12.52 MB                                                      
Downloaded  True                                                          

In [5]:
x, y = next(iter(dataset))
x, y

({'moment': datetime.datetime(2016, 4, 1, 0, 0, 7),
  'station': 'metro-canal-du-midi',
  'clouds': 75,
  'description': 'light rain',
  'humidity': 81,
  'pressure': 1017.0,
  'temperature': 6.54,
  'wind': 9.3},
 1)

In [6]:
from river import stream

X_y = stream.iter_csv(dataset.path)
x, y = next(X_y)
x, y

({'moment': '2016-04-01 00:00:07',
  'bikes': '1',
  'station': 'metro-canal-du-midi',
  'clouds': '75',
  'description': 'light rain',
  'humidity': '81',
  'pressure': '1017.0',
  'temperature': '6.54',
  'wind': '9.3'},
 None)

Come vediamo all'interno del nosto dizionario non abbiamo fatto il cast dei numeri ma abbiamo stringhe, infatti se carichiamo attraverso ```stream.inter_csv``` assumiamo che tutto sia a stringa, poi altro problema che possiamo avere è il fatto che il Moment non è in datetime ed infine abbiamo il valore fondamentale il valore bikes!

In [7]:
X_y = stream.iter_csv(
    dataset.path,
    converters={
        'bikes': int,
        'clouds': int,
        'humidity' : int,
        'pressure': float,
        'temperature': float,
        'wind': float
    },
    parse_dates={'moment': '%Y-%m-%d %H:%M:%S'},
    target='bikes'
)

x, y = next(X_y)
x,y

({'moment': datetime.datetime(2016, 4, 1, 0, 0, 7),
  'station': 'metro-canal-du-midi',
  'clouds': 75,
  'description': 'light rain',
  'humidity': 81,
  'pressure': 1017.0,
  'temperature': 6.54,
  'wind': 9.3},
 1)

Quindi River non è lo strumento migliore per il preporcessing, quindi magari sarebbe sempre meglio optare per soluzioni differenti, vediamo come anche la posizione potrebbe essere una features, e la descirzione del clima.... ci sono tante cose che possono essere migliorate!
Altra soluzione è quella di poter convertire anche altre tipologie di dataset in modo da poter essere lette in formato stream, per esempio abbiamo il modulo ```stream.iter_sklearn_dataset``` che permette di converitre un dataset scikit-learn dataset in uno stream.

In [8]:
from sklearn import datasets

dataset = datasets.load_diabetes()

for x, y in stream.iter_sklearn_dataset(dataset):
    break

x, y 


({'age': 0.0380759064334241,
  'sex': 0.0506801187398187,
  'bmi': 0.0616962065186885,
  'bp': 0.0218723549949558,
  's1': -0.0442234984244464,
  's2': -0.0348207628376986,
  's3': -0.0434008456520269,
  's4': -0.00259226199818282,
  's5': 0.0199084208763183,
  's6': -0.0176461251598052},
 151.0)

Dobbiamo fare una piccola zoomata sulla differenza tra **Proactive** learning e **Reactive Learning**, dove nel primo caso noi abbiamo già il nostro dataset e lo iteriamo tramite un ciclo For, quindi chiamiamo noi il tutto.

Dall'altro lato in una situazione di proactive learning invece non abbiamo controllo sull'arrivo dei dati, per esempio siamo in un contesto web server dove le web request arrivano in modo albitrario. Ed è qui che River brilla è può far vedere il suo meglio. 

Possiamo per esempio avere un applicazione che fa uso di Flask, e quindi possiamo definire u a strata al make prediction con river vediamo un esempio

In [12]:
import flask

app = flask.Flask(__name__)

@app.route('/', methods = ['GET'])
def predict():
    payload = flask.request.json
    river_model = load_model()
    return river_model.predict_proba_one(payload)

In [13]:
@app.route('/', methods=['POST'])
def learn():
    payload = flask.request.json
    river_model = load_model()
    river_model.learn_one(payload['features'], payload['target'])
    return {}, 201